<h1>ECOSISTEMA SPARK</h1>

<h1>Práctica 2: SparkSQL</h1>

<h1>TAREA 3</h1>

<h4>Autor: Guillermo García López</h4>

<h1><font color='gray'>Objetivo 1<font></h1>

In [1]:
# Comando para habilitar scrolling horizontal en celdas
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
# Se crea sparksession para utilidades SparkSQL
from pyspark.sql import SparkSession, Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL - Practica 2") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
# Se crea SparkContext() para utilidades sobre RDD's
sc = SparkContext.getOrCreate()

Se cargan los dataframes a partir de los csv's:

In [3]:
bookRatings = spark.read.option("delimiter", ";").option("header", "true")     \
                   .csv('./books_crossing/BX-CSV-Dump/BX-Book-Ratings.csv')
books = spark.read.option("delimiter", ";").option("header", "true")     \
             .csv('./books_crossing/BX-CSV-Dump/BX-Books.csv')
users = spark.read.option("delimiter", ";").option("header", "true")     \
                   .csv('./books_crossing/BX-CSV-Dump/BX-Users.csv')

In [4]:
bookRatings.limit(5).show()

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
+-------+----------+-----------+



In [5]:
books.limit(5).show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [6]:
users.limit(5).show()

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|NULL|
+-------+--------------------+----+



Y se calculan los datos pedidos:

- Lista de usuarios junto con el número de libros que han valorado:

In [7]:
bookRatings.groupBy(['User-ID']).count().withColumnRenamed('count', '#_books_rated').show()

+-------+-------------+
|User-ID|#_books_rated|
+-------+-------------+
| 277594|            1|
| 277840|            1|
| 278220|            2|
| 278659|            1|
|   1436|           12|
|   2136|           10|
|   3959|            2|
|   4032|            2|
|   4821|            3|
|   4937|            5|
|   5325|            1|
|   5925|            1|
|   6613|            1|
|   6731|           26|
|   7711|            1|
|   8433|            1|
|   9030|            1|
|   9583|            1|
|   9586|            2|
|   9993|            1|
+-------+-------------+
only showing top 20 rows



- Rating máximo recibido por cada editorial:

Se cruzan 'bookRatings' y 'books' por columna 'ISBN', deshechando filas con valor 'Publisher' a nulos, renombrando
las columnas 'Book-Rating' y 'Book-Author' para eliminar el carácter '-' y casteandola columna 'BookRating' 
a tipo 'int':

In [8]:
booksAndRatings = books.join(bookRatings, ['ISBN'], 'full').select('ISBN', 'Book-Rating', 'Publisher', 'Book-Author') \
                       .withColumnRenamed('Book-Rating', 'BookRating') \
                       .withColumnRenamed('Book-Author', 'BookAuthor') \
                       .selectExpr("cast(ISBN as string) ISBN", "cast(BookRating as integer) BookRating", \
                                   "cast(Publisher as string) Publisher", "cast(BookAuthor as string) BookAuthor")

In [9]:
booksAndRatings.printSchema()

root
 |-- ISBN: string (nullable = true)
 |-- BookRating: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- BookAuthor: string (nullable = true)



Y se realiza la operación para retornar los datos:

In [10]:
maxRatingperPubl = booksAndRatings.groupBy(['Publisher']) \
                                  .max('BookRating') \
                                  .withColumnRenamed('max(BookRating)', 'max_rate_obtained')

In [11]:
maxRatingperPubl.show()

+--------------------+-----------------+
|           Publisher|max_rate_obtained|
+--------------------+-----------------+
|Harper Mass Marke...|               10|
|Houghton Mifflin ...|               10|
|Carroll &amp; Gra...|               10|
|      Celestial Arts|               10|
|Faith Publishing ...|                8|
|         Cleis Press|               10|
|Ullstein-Taschenb...|               10|
|Chicago Review Press|               10|
|    Adams Media Corp|               10|
|         Bertelsmann|                9|
|Orion Business Books|                1|
|        Book Pub. Co|                0|
|        Chosen Books|               10|
|         Hermetic Pr|               10|
|        Lorenz Books|               10|
| Research Press (IL)|                7|
|Editiones B, Grup...|                0|
|      Aqua Explorers|                9|
|           LPC Group|                9|
|Community Communi...|                8|
+--------------------+-----------------+
only showing top

- Nombre del autor que ha recibido más ratings:

Con el dataframe anteriormente creado, podemos realizar la operación de agregación para obtener el resultado:

In [13]:
mostRatedAuthor = booksAndRatings.groupBy(['BookAuthor']).count().withColumnRenamed('count', '#_ratings') \
                                 .orderBy(['#_ratings'], ascending=0)

In [14]:
mostRatedAuthor.where(mostRatedAuthor['BookAuthor'] != 'null').limit(1).show()

+------------+---------+
|  BookAuthor|#_ratings|
+------------+---------+
|Stephen King|    10055|
+------------+---------+



De manera que el autor que más ratings ha recibido es Stephen King.

<h1><font color='gray'>Objetivo 2<font></h1>